In [1]:
import string
import random
from PIL import Image, ImageOps, ImageDraw, ImageFont, ImageChops
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from img_processing import preprocess_img, DITHER_MODES
from ascii_art_non_mono_utils import *
from ascii_art_non_mono_algorithms import *

In [2]:
font_path = "fonts/OpenSans-Regular.ttf"
size = 20
fnt = ImageFont.truetype(font_path, size)

In [3]:
orig_img = Image.open("imgs/irad_grad.bmp").convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [4]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, 0.3, 1.1, 0.9, 0.2, 128, DITHER_MODES.NONE)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [5]:
proc_img = interactive_preprocess.result

In [6]:
palette = list(filter(lambda a: a.isprintable(), string.printable))
print(len(palette))
print(palette)

95
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ']


In [7]:
lrs = lazy_random_search(proc_img, palette, fnt)

for i in range(0, 100):
    text = next(lrs)
    if i % 100 == 0:
        print(evaluate_text_arr(text, proc_img, fnt))

bg_img, draw = new_img_draw(proc_img.size)
draw.multiline_text((0,0), ''.join(text), font=fnt, fill=255)
ImageChops.add(proc_img, bg_img)

-105.13202029758321


In [8]:
text = greedy_algorithm(proc_img, palette, fnt)

print(''.join(text))

bg_img, draw = new_img_draw(proc_img.size)
draw.multiline_text((0,0), ''.join(text), font=fnt, fill=255)
print(evaluate_text_arr(text, proc_img, fnt))
ImageChops.add(proc_img, bg_img)

       jjjjjjjjjjjjjjjj|~
   jjjjJJJJJJJJJJJJJJJ@m
 jjjjJJJJJJJJJJJJJJJJ@@
 lljjJJJJJJJJJJJJJJJJ@@
  'lllljJJJJJJJJJJJJJJ@m
     '''''''llllllllllll~~

-93.31857937980386


In [9]:
lines = split_lines(proc_img, palette, fnt)
line = lines[len(lines)//2]

harmony_line_search = lazy_generate_harmony_line(line, palette, fnt, 5, 0.7, 0.3, len(palette))

In [10]:
for _ in range(10000):
    next(harmony_line_search)
next_harmony_solution, fit = next(harmony_line_search)
print(fit)
text = palette_id_arr_to_text_arr(next_harmony_solution, palette)
bg_img, draw = new_img_draw(line.size)
draw.text((0,0), ''.join(text), font=fnt, fill=255)
ImageChops.add(line, bg_img)

1000 -62.17116564417178 -62.6345797231268
2000 -60.75563421828909 -61.76420556998979
3000 -60.75563421828909 -61.4604121587794
4000 -58.51733333333333 -60.29123679338085
5000 -58.51733333333333 -60.01475473223902
6000 -58.28660968660969 -58.729274177186184
7000 -58.28660968660969 -58.729274177186184
8000 -56.058836565096954 -58.074471900339084
9000 -56.058836565096954 -58.074471900339084
10000 -56.058836565096954 -57.918980632519414
-56.058836565096954
